In [ ]:
import pulsenet as pn
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models

In [ ]:
model_fine_tuned = pn.LoadModel("trace_autoencoder_fine_tuned.h5", "phase_regressor_fine_tuned.h5", "amplitude_regressor_fine_tuned.h5", "pileup_classifier_fine_tuned.h5")
model_transfer_learned = pn.LoadModel("trace_autoencoder_transfer_learned.h5", "phase_regressor_initial.h5", "amplitude_regressor_initial.h5", "pileup_classifier_initial.h5")
model_initial = pn.LoadModel("trace_autoencoder_only_pileup.h5", "phase_regressor_initial.h5", "amplitude_regressor_initial.h5", "pileup_classifier_initial.h5")

In [ ]:
filename = "ysoTracesNoPileup.root"
x_trace, y_trace, y_phase, y_amp =  pn.CreateMockData(filename, pileup_split=0.5, phase_min=0, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
y_pileup_onehot = pn.OneHotEncodePileup(y_phase)   

In [ ]:
# predict on data
trace_pred_fine_tuned, phase_pred_fine_tuned, amp_pred_fine_tuned, pileup_pred_fine_tuned = model_fine_tuned.predict(x_trace)
trace_pred_transfer_learned, phase_pred_transfer_learned, amp_pred_transfer_learned, pileup_pred_transfer_learned = model_transfer_learned.predict(x_trace)
trace_pred_initial, phase_pred_initial, amp_pred_initial, pileup_pred_initial = model_initial.predict(x_trace)

In [ ]:
# evaluate the model
performance_fine_tuned = model_fine_tuned.evaluate(x_trace, [y_trace, y_phase, y_amp, y_pileup_onehot], verbose=0)
performance_transfer_learned = model_transfer_learned.evaluate(x_trace, [y_trace, y_phase, y_amp, y_pileup_onehot], verbose=0)
performance_initial = model_initial.evaluate(x_trace, [y_trace, y_phase, y_amp, y_pileup_onehot], verbose=0)

In [ ]:
# calulate residuals
phase_fine_tuned_residuals = phase_pred_fine_tuned - y_phase
phase_transfer_learned_residuals = phase_pred_transfer_learned - y_phase
phase_initial_residuals = phase_pred_initial - y_phase

amp_fine_tuned_residuals = amp_pred_fine_tuned - y_amp
amp_transfer_learned_residuals = amp_pred_transfer_learned - y_amp
amp_initial_residuals = amp_pred_initial - y_amp

In [ ]:
from sklearn.metrics import roc_curve
pred_class_fine_tuned = pileup_pred_fine_tuned[:,0] # pileup classifier output is (1,0) if pileup, (0,1) if not pileup
pred_class_transfer_learned = pileup_pred_transfer_learned[:,0]
pred_class_initial = pileup_pred_initial[:,0]
truth_class = y_pileup_onehot[:,0] # one hot encoded truth is (1,0) if pileup, (0,1) if not pileup

ft_fpr, ft_tpr, ft_thresholds = roc_curve(truth_class, pred_class_fine_tuned)
tl_fpr, tl_tpr, tl_thresholds = roc_curve(truth_class, pred_class_transfer_learned)
in_fpr, in_tpr, in_thresholds = roc_curve(truth_class, pred_class_initial)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ft_fpr, ft_tpr, label="Fine Tuned")
plt.plot(tl_fpr, tl_tpr, label="Transfer Learned")
plt.plot(in_fpr, in_tpr, label="Initial")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label="No Skill")
plt.legend()

In [ ]:
import numpy as np
import pandas as pd
phase_bins = np.linspace(0,20,20)
amp_bins = np.linspace(0.5,1.5,20)

pdf = pd.DataFrame({"Phase":y_phase[:,0], "Phase Fine Tuned":phase_pred_fine_tuned[:,0], "Phase Transfer Learned":phase_pred_transfer_learned[:,0], "Phase Initial":phase_pred_initial[:,0]})
adf = pd.DataFrame({"Amplitude":y_amp[:,0], "Amplitude Fine Tuned":amp_pred_fine_tuned[:,0], "Amplitude Transfer Learned":amp_pred_transfer_learned[:,0], "Amplitude Initial":amp_pred_initial[:,0]})

p_ft_sigma = []
p_tl_sigma = []
p_in_sigma = []
a_ft_sigma = []
a_tl_sigma = []
a_in_sigma = []


for i in range(0,19):
    pdf_bin = pdf[(pdf["Phase"] > phase_bins[i]) & (pdf["Phase"] < phase_bins[i+1])].copy()
    adf_bin = adf[(adf["Amplitude"] > amp_bins[i]) & (adf["Amplitude"] < amp_bins[i+1])].copy()
    p_ft_sigma.append(pdf_bin["Phase Fine Tuned"].std())
    p_tl_sigma.append(pdf_bin["Phase Transfer Learned"].std())
    p_in_sigma.append(pdf_bin["Phase Initial"].std())
    a_ft_sigma.append(adf_bin["Amplitude Fine Tuned"].std())
    a_tl_sigma.append(adf_bin["Amplitude Transfer Learned"].std())
    a_in_sigma.append(adf_bin["Amplitude Initial"].std())   


In [ ]:
plt.plot(phase_bins[:-1], p_ft_sigma, label="Fine Tuned")
plt.plot(phase_bins[:-1], p_tl_sigma, label="Transfer Learned")
plt.plot(phase_bins[:-1], p_in_sigma, label="Initial")
plt.xlabel(r"Phase $\phi$ (ns)")
plt.ylabel(r"$\sigma(\delta\phi)$")
plt.legend()

In [ ]:
plt.plot(amp_bins[:-1], a_ft_sigma, label="Fine Tuned")
plt.plot(amp_bins[:-1], a_tl_sigma, label="Transfer Learned")
plt.plot(amp_bins[:-1], a_in_sigma, label="Initial")
plt.xlabel(r"Amplitude $A$ (mV)")
plt.ylabel(r"$\sigma(\delta A)$")
plt.legend()

In [ ]:
def KL_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

ft_kl0 = KL_divergence(trace_pred_fine_tuned[:,0], y_trace[:,0])
tl_kl0= KL_divergence(trace_pred_transfer_learned[:,0], y_trace[:,0])   
in_kl0 = KL_divergence(trace_pred_initial[:,0], y_trace[:,0])

ft_kl1 = KL_divergence(phase_pred_fine_tuned[:,0], y_phase[:,0])
tl_kl1 = KL_divergence(phase_pred_transfer_learned[:,0], y_phase[:,0])
in_kl1 = KL_divergence(phase_pred_initial[:,0], y_phase[:,0])

ft_kl = np.concatenate((ft_kl0, ft_kl1), axis=0)
tl_kl = np.concatenate((tl_kl0, tl_kl1), axis=0)
in_kl = np.concatenate((in_kl0, in_kl1), axis=0)

In [ ]:
plt.hist(ft_kl, bins=100, label="Fine Tuned", alpha=0.5, density=True)
plt.hist(tl_kl, bins=100, label="Transfer Learned", alpha=0.5, density=True)
plt.hist(in_kl, bins=100, label="Initial", alpha=0.5, density=True)
plt.xlabel("KL Divergence")
plt.ylabel("Count")
plt.legend()